# TensorFlow + Keras Full topic covers (Short Practical)


 Tensor basics • Autograd • Keras training • tf.data • CNN • Transfer Learning • RNN • Custom training loop • Callbacks • TensorBoard • Save/Load • Inference • TFLite export




## 0) Install / Check TensorFlow

In [1]:
import tensorflow as tf
print('TF Version:', tf.__version__)

TF Version: 2.20.0


## 1) Tensor Basics (constant, variable, dtype, shape)

In [2]:
import numpy as np
import tensorflow as tf

a = tf.constant([[1,2],[3,4]], dtype=tf.float32)
b = tf.Variable([[10,20],[30,40]], dtype=tf.float32)

print("a:", a)
print("b:", b)
print("shape:", a.shape, "dtype:", a.dtype)


a: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
b: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[10., 20.],
       [30., 40.]], dtype=float32)>
shape: (2, 2) dtype: <dtype: 'float32'>


## 2) Basic Tensor Operations (add, mul, matmul)

In [3]:
x = tf.constant([[1.,2.],[3.,4.]])
y = tf.constant([[5.,6.],[7.,8.]])

print("add:", x + y)
print("mul:", x * y)
print("matmul:", tf.matmul(x, y))


add: tf.Tensor(
[[ 6.  8.]
 [10. 12.]], shape=(2, 2), dtype=float32)
mul: tf.Tensor(
[[ 5. 12.]
 [21. 32.]], shape=(2, 2), dtype=float32)
matmul: tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


## 3) Reshape / Expand / Squeeze

In [4]:
t = tf.range(12)
t2 = tf.reshape(t, (3,4))
t3 = tf.expand_dims(t2, axis=0)
t4 = tf.squeeze(t3)

print("t:", t.shape)
print("reshape:", t2.shape)
print("expand:", t3.shape)
print("squeeze:", t4.shape)


t: (12,)
reshape: (3, 4)
expand: (1, 3, 4)
squeeze: (3, 4)


## 4) NumPy ↔ TensorFlow Conversion

In [5]:
np_arr = np.array([1,2,3])
t = tf.convert_to_tensor(np_arr)
back = t.numpy()

print("Tensor:", t)
print("Back to numpy:", back)


Tensor: tf.Tensor([1 2 3], shape=(3,), dtype=int64)
Back to numpy: [1 2 3]


## 5) Autograd (tf.GradientTape)

In [6]:
w = tf.Variable(3.0)

with tf.GradientTape() as tape:
    loss = w**2 + 2*w + 1   # (w+1)^2

grad = tape.gradient(loss, w)
print("loss:", float(loss), "grad:", float(grad))


loss: 16.0 grad: 8.0


## 6) Simple Linear Regression with GradientTape (Manual Training Loop)

In [7]:
# y = 3x + 2 + noise
X = tf.random.normal((200, 1))
y = 3*X + 2 + tf.random.normal((200,1))*0.2

W = tf.Variable(tf.random.normal((1,1)))
b = tf.Variable(tf.zeros((1,)))

opt = tf.keras.optimizers.SGD(learning_rate=0.1)

for epoch in range(30):
    with tf.GradientTape() as tape:
        y_pred = tf.matmul(X, W) + b
        loss = tf.reduce_mean((y - y_pred)**2)
    dW, db = tape.gradient(loss, [W, b])
    opt.apply_gradients(zip([dW, db], [W, b]))

print("Trained W:", W.numpy().ravel(), "b:", b.numpy())


Trained W: [3.0155318] b: [1.9834859]


## 7) Keras Basics (Sequential model)

In [8]:
from tensorflow.keras import layers, Sequential

model = Sequential([
    layers.Dense(32, activation="relu", input_shape=(10,)),
    layers.Dense(3, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


c:\Users\Manish Mishra\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 451 (1.76 KB)

 Trainable params: 451 (1.76 KB)

 Non-trainable params: 0 (0.00 B)

## 8) Training / Evaluate / Predict (Keras workflow)

In [9]:
X = np.random.randn(300, 10).astype(np.float32)
y = np.random.randint(0, 3, size=(300,))

history = model.fit(X, y, epochs=3, batch_size=32, validation_split=0.2)

loss, acc = model.evaluate(X, y, verbose=0)
print("loss:", loss, "acc:", acc)

pred = model.predict(X[:2])
print("pred shape:", pred.shape)


Epoch 1/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3083 - loss: 1.2198 - val_accuracy: 0.3333 - val_loss: 1.2064
Epoch 2/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3125 - loss: 1.1921 - val_accuracy: 0.3500 - val_loss: 1.1957
Epoch 3/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3250 - loss: 1.1713 - val_accuracy: 0.3667 - val_loss: 1.1869
loss: 1.1644476652145386 acc: 0.33666667342185974
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
pred shape: (2, 3)


## 9) tf.data Pipeline (shuffle, batch, prefetch)

In [10]:
ds = tf.data.Dataset.from_tensor_slices((X, y))
ds = ds.shuffle(300).batch(32).prefetch(tf.data.AUTOTUNE)

for xb, yb in ds.take(1):
    print("batch X:", xb.shape, "batch y:", yb.shape)


batch X: (32, 10) batch y: (32,)


## 10) Callbacks (EarlyStopping + ReduceLR + Checkpoint)

In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
    tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss")
]
print("Callbacks ready ")


Callbacks ready 


## 11) CNN Basics (Conv2D + Pooling + Dropout)

In [13]:
cnn = tf.keras.Sequential([
    layers.Conv2D(32, 3, activation="relu", input_shape=(64,64,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(5, activation="softmax")
])

cnn.compile(optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])

cnn.summary()


c:\Users\Manish Mishra\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 822,597 (3.14 MB)

 Trainable params: 822,597 (3.14 MB)

 Non-trainable params: 0 (0.00 B)

## 12) Image Dataset Loader (image_dataset_from_directory template)

In [ ]:
# IMG_SIZE = (128, 128)
# BATCH_SIZE = 32
# train_ds = tf.keras.utils.image_dataset_from_directory(
#     "path/train",
#     image_size=IMG_SIZE,
#     batch_size=BATCH_SIZE
# )
# val_ds = tf.keras.utils.image_dataset_from_directory(
#     "path/val",
#     image_size=IMG_SIZE,
#     batch_size=BATCH_SIZE
# )
# train_ds = train_ds.map(lambda x,y: (x/255.0, y)).prefetch(tf.data.AUTOTUNE)
# val_ds   = val_ds.map(lambda x,y: (x/255.0, y)).prefetch(tf.data.AUTOTUNE)


## 13) Data Augmentation (Keras layers)

In [14]:
augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

dummy = tf.random.uniform((1, 64, 64, 3))
aug = augment(dummy)
print("Augmented shape:", aug.shape)


Augmented shape: (1, 64, 64, 3)


## 14) Transfer Learning (MobileNetV2)

In [15]:
base = tf.keras.applications.MobileNetV2(
    include_top=False, weights="imagenet", input_shape=(128,128,3)
)
base.trainable = False

tl_model = tf.keras.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(5, activation="softmax")
])

tl_model.compile(optimizer="adam",
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])

tl_model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,264,389 (8.64 MB)

 Trainable params: 6,405 (25.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## 15) Fine-Tuning (Unfreeze last layers)

In [16]:
# Unfreeze small part of base model
base.trainable = True
for layer in base.layers[:-20]:
    layer.trainable = False

tl_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])

print("Fine-tuning enabled ")


Fine-tuning enabled 


## 16) RNN / LSTM (Text or sequence)

In [17]:
# Fake sequence data: (batch, timesteps, features)
X_seq = np.random.randn(200, 30, 8).astype(np.float32)
y_seq = np.random.randint(0, 2, size=(200,))

rnn_model = tf.keras.Sequential([
    layers.LSTM(32, input_shape=(30, 8)),
    layers.Dense(1, activation="sigmoid")
])

rnn_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
rnn_model.fit(X_seq, y_seq, epochs=2, batch_size=16, validation_split=0.2)


Epoch 1/2


c:\Users\Manish Mishra\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.5188 - loss: 0.6913 - val_accuracy: 0.5500 - val_loss: 0.6845
Epoch 2/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5625 - loss: 0.6856 - val_accuracy: 0.6000 - val_loss: 0.6824


## 17) Custom Loss Function

In [25]:
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    smooth = 1e-6
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return 1 - (2.0 * intersection + smooth) / (union + smooth)

print("Dice loss ready ")


Dice loss ready 


## 18) Custom Training Step (Subclass Model)

In [19]:
class CustomModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = layers.Dense(32, activation="relu")
        self.out = layers.Dense(3, activation="softmax")

    def call(self, x):
        x = self.d1(x)
        return self.out(x)

custom = CustomModel()
custom.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
custom.fit(X, y, epochs=2, batch_size=32)


Epoch 1/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3367 - loss: 1.1957  
Epoch 2/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3300 - loss: 1.1698 


## 19) TensorBoard Logging

In [26]:
# logs_dir = "logs"
# tb = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
# model.fit(X, y, epochs=3, callbacks=[tb])
print("TensorBoard template ready ")


TensorBoard template ready 


## 20) Save / Load Model (.keras and .h5)

In [22]:
model.save("model_demo.keras")
loaded1 = tf.keras.models.load_model("model_demo.keras")

model.save("model_demo.h5")  # legacy
loaded2 = tf.keras.models.load_model("model_demo.h5")

print("Saved & loaded successfully ")


c:\Users\Manish Mishra\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Saved & loaded successfully 


## 21) Inference (Predict single image path)

In [23]:
import numpy as np

def predict_single_image(model, img_path, img_size=(128,128)):
    img = tf.keras.utils.load_img(img_path, target_size=img_size)
    arr = tf.keras.utils.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    probs = model.predict(arr)[0]
    return np.argmax(probs), probs

print("Inference function ready ")


Inference function ready 


## 22) Export to TensorFlow Lite (TFLite)

In [24]:
# Convert any keras model to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite saved as model.tflite ")


INFO:tensorflow:Assets written to: C:\Users\MANISH~1\AppData\Local\Temp\tmpx9k6tf08\assets


INFO:tensorflow:Assets written to: C:\Users\MANISH~1\AppData\Local\Temp\tmpx9k6tf08\assets


Saved artifact at 'C:\Users\MANISH~1\AppData\Local\Temp\tmpx9k6tf08'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2778572088784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2778572089936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2778572090320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2778572084368: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite saved as model.tflite 
